# ETL Operations in Python Sample #
*Written by Hassan Syyid @ [hotglue](https://hotglue.xyz)*

Check out the corresponding [Medium Article]()
## Introduction ##
This is a brief sample showing how Python can be used effectively for ETL operations. We'll go through the process of cleaning up raw data and making simple transformations (**filtering**, **exploding**, **flattening**, **reducing**, etc.). These are typical operations used for data integration.

This example leverages sample Quickbooks data from the Quickbooks Sandbox environment, and was initially created in a [hotglue](https://hotglue.xyz) environment - a light-weight data integration tool for startups.

In [104]:
import ast
import gluestick as gs
import pandas as pd

### Step 1: Read the data ###
Let's start by reading the data. 

This example is built on a hotglue environment with data coming from Quickbooks. In hotglue, the data is placed in the local `sync-output` folder in a CSV format. We will use the [gluestick](https://pypi.org/project/gluestick/) package to read the raw data in the input folder into a dictionary of pandas dataframes using the `read_csv_folder` function.

By specifying `index_cols={'Invoice': 'DocNumber'}` the `Invoices` dataframe will use the `DocNumber` column as an index.
By specifying converters, we can use `ast` to parse the JSON data in the `Line` and `CustomField` columns.

In [105]:
# standard directory for hotglue
ROOT_DIR = "./sync-output"

# Read input data
input_data = gs.read_csv_folder(ROOT_DIR,
                                index_cols={'Invoice': 'DocNumber'},
                                converters={'Invoice': {'Line': ast.literal_eval, 'CustomField': ast.literal_eval,
                                                        'Categories': ast.literal_eval}})

##### Take a peek #####
Let's take a look at what data we're working with. For simplicity, I've selected the columns I'd like to work with and saved it to `input_df`. Typically in hotglue you can configure this using a field map, but I've done it manually here.

In [106]:
input_df = input_data['Invoice'][['Id', 'CustomerRef__value', 'CustomerRef__name', 'MetaData__LastUpdatedTime', 'MetaData__CreateTime', 'CurrencyRef__name', 'CurrencyRef__value', 'Line', 'CustomField']]
input_df.head()

,Id,CustomerRef__value,CustomerRef__name,MetaData__LastUpdatedTime,MetaData__CreateTime,CurrencyRef__name,CurrencyRef__value,Line,CustomField
DocNumber,,,,,,,,,
1037,130,24,Sonnenschein Family Store,2020-06-20T20:16:17.000000Z,2020-06-20T20:16:17.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 275.0, ...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1036,129,8,0969 Ocean View Road,2020-06-20T20:15:36.000000Z,2020-06-20T20:15:36.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 50.0, '...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1031,96,8,0969 Ocean View Road,2020-06-20T20:13:33.000000Z,2020-06-19T20:30:49.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 90.0, '...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1004,12,3,Cool Cars,2020-06-20T19:59:21.000000Z,2020-06-17T22:04:04.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 20.0, '...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1035,119,17,Mark Cho,2020-06-20T19:57:24.000000Z,2020-06-20T19:57:24.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 275.0, ...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."


### Step 2: Rename Columns ###
Let's clean up the data by renaming the columns to more readable names.
```
CustomerRef__value -> CustomerId
CustomerRef__name -> Customer
MetaData_LastUpdatedTime -> LastUpdated
MetaData_CreateTime -> CreatedOn
CurrencyRef__name -> Currency
CurrencyRef__value -> CurrencyCode
```

In [107]:
# Let's clean up the names of these columns
invoices = input_df.pipe(lambda x: x.rename(columns={'CustomerRef__value': 'CustomerId', 'CustomerRef__name': 'Customer',
                                              'MetaData__LastUpdatedTime': 'LastUpdated',
                                              'MetaData__CreateTime': 'CreatedOn', 'CurrencyRef__name': 'Currency',
                                              'CurrencyRef__value': 'CurrencyCode'}))
invoices.head()

,Id,CustomerId,Customer,LastUpdated,CreatedOn,Currency,CurrencyCode,Line,CustomField
DocNumber,,,,,,,,,
1037,130,24,Sonnenschein Family Store,2020-06-20T20:16:17.000000Z,2020-06-20T20:16:17.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 275.0, ...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1036,129,8,0969 Ocean View Road,2020-06-20T20:15:36.000000Z,2020-06-20T20:15:36.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 50.0, '...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1031,96,8,0969 Ocean View Road,2020-06-20T20:13:33.000000Z,2020-06-19T20:30:49.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 90.0, '...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1004,12,3,Cool Cars,2020-06-20T19:59:21.000000Z,2020-06-17T22:04:04.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 20.0, '...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1035,119,17,Mark Cho,2020-06-20T19:57:24.000000Z,2020-06-20T19:57:24.000000Z,United States Dollar,USD,"[{'Id': '1', 'LineNum': '1', 'Amount': 275.0, ...","[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."


### Step 3: Extract information ###

The `Line` column is actually a serialized JSON object provided by Quickbooks with several useful elements in it. We'll need to start by **flattening** the JSON and then **exploding** into unique columns so we can work with the data.

Again, we'll use the [gluestick](https://pypi.org/project/gluestick/) package to accomplish this. The `explode_json_to_rows` function handles the flattening and exploding in one step. To avoid exploding too many levels of this object, we'll specify `max_level=1`

Here is a snippet from one to give you an idea.
```json
[{
	'Id': '1',
	'LineNum': '1',
	'Amount': 275.0,
	'DetailType': 'SalesItemLineDetail',
	'SalesItemLineDetail': {
		'ItemRef': {
			'value': '5',
			'name': 'Rock Fountain'
		},
		'ItemAccountRef': {
			'value': '79',
			'name': 'Sales of Product Income'
		},
		'TaxCodeRef': {
			'value': 'TAX',
			'name': None
		}
	},
	'SubTotalLineDetail': None,
	'DiscountLineDetail': None
}]
```

In [108]:
# Let's explode the Line column now
invoices = invoices.pipe(gs.explode_json_to_rows, "Line", max_level=1)
invoices.head()

,Id,CustomerId,Customer,LastUpdated,CreatedOn,Currency,CurrencyCode,CustomField,Line.Id,Line.LineNum,Line.Amount,Line.DetailType,Line.SubTotalLineDetail,Line.DiscountLineDetail,Line.SalesItemLineDetail.ItemRef,Line.SalesItemLineDetail.ItemAccountRef,Line.SalesItemLineDetail.TaxCodeRef,Line.SalesItemLineDetail,Line.DiscountLineDetail.DiscountAccountRef
DocNumber,,,,,,,,,,,,,,,,,,,
1001,9,1,Amy's Bird Sanctuary,2020-06-19T20:25:20.000000Z,2020-06-17T21:49:30.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",1,1,100.0,SalesItemLineDetail,NaN,NaN,"{'value': '6', 'name': 'Gardening'}","{'value': '45', 'name': 'Landscaping Services'}","{'value': 'TAX', 'name': None}",NaN,NaN
1001,9,1,Amy's Bird Sanctuary,2020-06-19T20:25:20.000000Z,2020-06-17T21:49:30.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",None,None,100.0,SubTotalLineDetail,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,10,2,Bill's Windsurf Shop,2020-06-19T19:56:01.000000Z,2020-06-17T21:57:16.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",1,1,140.0,SalesItemLineDetail,NaN,NaN,"{'value': '6', 'name': 'Gardening'}","{'value': '45', 'name': 'Landscaping Services'}","{'value': 'NON', 'name': None}",NaN,NaN
1002,10,2,Bill's Windsurf Shop,2020-06-19T19:56:01.000000Z,2020-06-17T21:57:16.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",2,2,35.0,SalesItemLineDetail,NaN,NaN,"{'value': '10', 'name': 'Pest Control'}","{'value': '54', 'name': 'Pest Control Services'}","{'value': 'NON', 'name': None}",NaN,NaN
1002,10,2,Bill's Windsurf Shop,2020-06-19T19:56:01.000000Z,2020-06-17T21:57:16.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",None,None,175.0,SubTotalLineDetail,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 4: Filter rows ###

For our purposes, we only want to work with rows with a `Line.DetailType` of `SalesItemLineDetail` (we dont need sub-total lines). This is a common ETL operation known as **filtering** and is accomplished easily with pandas

In [109]:
# We filter for only SalesItemLineDetail
invoices = invoices.pipe(lambda x: x[x['Line.DetailType'] == 'SalesItemLineDetail'])
invoices.head()

,Id,CustomerId,Customer,LastUpdated,CreatedOn,Currency,CurrencyCode,CustomField,Line.Id,Line.LineNum,Line.Amount,Line.DetailType,Line.SubTotalLineDetail,Line.DiscountLineDetail,Line.SalesItemLineDetail.ItemRef,Line.SalesItemLineDetail.ItemAccountRef,Line.SalesItemLineDetail.TaxCodeRef,Line.SalesItemLineDetail,Line.DiscountLineDetail.DiscountAccountRef
DocNumber,,,,,,,,,,,,,,,,,,,
1001,9,1,Amy's Bird Sanctuary,2020-06-19T20:25:20.000000Z,2020-06-17T21:49:30.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",1,1,100.0,SalesItemLineDetail,NaN,NaN,"{'value': '6', 'name': 'Gardening'}","{'value': '45', 'name': 'Landscaping Services'}","{'value': 'TAX', 'name': None}",NaN,NaN
1002,10,2,Bill's Windsurf Shop,2020-06-19T19:56:01.000000Z,2020-06-17T21:57:16.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",1,1,140.0,SalesItemLineDetail,NaN,NaN,"{'value': '6', 'name': 'Gardening'}","{'value': '45', 'name': 'Landscaping Services'}","{'value': 'NON', 'name': None}",NaN,NaN
1002,10,2,Bill's Windsurf Shop,2020-06-19T19:56:01.000000Z,2020-06-17T21:57:16.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",2,2,35.0,SalesItemLineDetail,NaN,NaN,"{'value': '10', 'name': 'Pest Control'}","{'value': '54', 'name': 'Pest Control Services'}","{'value': 'NON', 'name': None}",NaN,NaN
1004,12,3,Cool Cars,2020-06-20T19:59:21.000000Z,2020-06-17T22:04:04.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",1,1,20.0,SalesItemLineDetail,NaN,NaN,"{'value': '16', 'name': 'Sprinkler Heads'}","{'value': '79', 'name': 'Sales of Product Inco...","{'value': 'TAX', 'name': None}",NaN,NaN
1004,12,3,Cool Cars,2020-06-20T19:59:21.000000Z,2020-06-17T22:04:04.000000Z,United States Dollar,USD,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type...",2,2,24.0,SalesItemLineDetail,NaN,NaN,"{'value': '17', 'name': 'Sprinkler Pipes'}","{'value': '79', 'name': 'Sales of Product Inco...","{'value': 'TAX', 'name': None}",NaN,NaN


### Step 5: More exploding ###
Look at some of the entries from the `Line` column we exploded. You'll notice they are name value pairs in JSON. 

Let's use [gluestick](https://pypi.org/project/gluestick/) again to explode these into new columns via the `json_tuple_to_cols` function. We'll need to specify `lookup_keys` - in our case, the `key_prop=name` and `value_prop=value` 

In [110]:
# Look at the columns we created from Line
invoices[['Id', 'Line.SalesItemLineDetail.ItemRef', 'Line.SalesItemLineDetail.ItemAccountRef']].head()

,Id,Line.SalesItemLineDetail.ItemRef,Line.SalesItemLineDetail.ItemAccountRef
DocNumber,,,
1001,9,"{'value': '6', 'name': 'Gardening'}","{'value': '45', 'name': 'Landscaping Services'}"
1002,10,"{'value': '6', 'name': 'Gardening'}","{'value': '45', 'name': 'Landscaping Services'}"
1002,10,"{'value': '10', 'name': 'Pest Control'}","{'value': '54', 'name': 'Pest Control Services'}"
1004,12,"{'value': '16', 'name': 'Sprinkler Heads'}","{'value': '79', 'name': 'Sales of Product Inco..."
1004,12,"{'value': '17', 'name': 'Sprinkler Pipes'}","{'value': '79', 'name': 'Sales of Product Inco..."


In [111]:
# Specify lookup keys
qb_lookup_keys = {'key_prop': 'name', 'value_prop': 'value'}

# Explode these into new columns
invoices = (invoices.pipe(gs.json_tuple_to_cols, 'Line.SalesItemLineDetail.ItemRef',
                  col_config={'cols': {'key_prop': 'Item', 'value_prop': 'Item Id'},
                              'look_up': qb_lookup_keys})
                  .pipe(gs.json_tuple_to_cols, 'Line.SalesItemLineDetail.ItemAccountRef',
                  col_config={'cols': {'key_prop': 'Item Ref', 'value_prop': 'Item Ref Id'},
                              'look_up': qb_lookup_keys}))
invoices[['Id', 'Item', 'Item Id', 'Item Ref', 'Item Ref Id']].head()

,Id,Item,Item Id,Item Ref,Item Ref Id
DocNumber,,,,,
1001,9,Gardening,6,Landscaping Services,45
1002,10,Gardening,6,Landscaping Services,45
1002,10,Pest Control,10,Pest Control Services,54
1004,12,Sprinkler Heads,16,Sales of Product Income,79
1004,12,Sprinkler Pipes,17,Sales of Product Income,79


### Step 6: Some more exploding ###
Take a look at the `CustomField` column. Below is an example of an entry

```json
[{'DefinitionId': '1', 'Name': 'Crew #', 'Type': 'StringType', 'StringValue': '102'}]
```

You can see this is JSON encoded data, specifying one custom field: `Crew #` with value `102`

To explode this, we'll need to **reduce** this as we only care about the `Name` and `StringValue`. We can use gluestick's `explode_json_to_cols` function with an `array_to_dict_reducer` to accomplish this. 

In [112]:
# Look at the CustomField column
invoices[['Id', 'CustomField']].head()

,Id,CustomField
DocNumber,,
1001,9,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1002,10,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1002,10,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1004,12,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."
1004,12,"[{'DefinitionId': '1', 'Name': 'Crew #', 'Type..."


In [113]:
# Grab the string value of entries
invoices = invoices.pipe(gs.explode_json_to_cols, 'CustomField', reducer=gs.array_to_dict_reducer('Name', 'StringValue'))
invoices[['Id', 'CustomField.Crew #']].head()

,Id,CustomField.Crew #
DocNumber,,
1001,9,None
1002,10,103
1002,10,103
1004,12,None
1004,12,None


## Conclusion ##
Our final data looks something like below. In this sample we went through several basic ETL operations using a real world example all with basic Python tools. Feel free to check out the open source [hotglue recipes](https://github.com/hotgluexyz/recipes) for more samples in the future.

In [114]:
invoices[['Id', 'CustomerId', 'Customer','LastUpdated','CreatedOn', 'CurrencyCode', 'Item', 'Item Id', 'Item Ref', 'Item Ref Id', 'CustomField.Crew #']].head()

,Id,CustomerId,Customer,LastUpdated,CreatedOn,CurrencyCode,Item,Item Id,Item Ref,Item Ref Id,CustomField.Crew #
DocNumber,,,,,,,,,,,
1001,9,1,Amy's Bird Sanctuary,2020-06-19T20:25:20.000000Z,2020-06-17T21:49:30.000000Z,USD,Gardening,6,Landscaping Services,45,None
1002,10,2,Bill's Windsurf Shop,2020-06-19T19:56:01.000000Z,2020-06-17T21:57:16.000000Z,USD,Gardening,6,Landscaping Services,45,103
1002,10,2,Bill's Windsurf Shop,2020-06-19T19:56:01.000000Z,2020-06-17T21:57:16.000000Z,USD,Pest Control,10,Pest Control Services,54,103
1004,12,3,Cool Cars,2020-06-20T19:59:21.000000Z,2020-06-17T22:04:04.000000Z,USD,Sprinkler Heads,16,Sales of Product Income,79,None
1004,12,3,Cool Cars,2020-06-20T19:59:21.000000Z,2020-06-17T22:04:04.000000Z,USD,Sprinkler Pipes,17,Sales of Product Income,79,None
